# PCA를 이용한 데이타 차원 축소

데이터가 포함하고 있는 변수의 숫자를 줄여야 할 경우가 많이 발생한다. 특히 변수에 숫자가 매우 많아서 이를 컴퓨터가 처리하기가 어려운 경우에는 컴퓨터가 다루는 변수 숫자를 줄여 주어야 한다. 알고리즘 중에는 변수의 숫자가 적어야만 동작을 잘하는 알고리즘이 많다. 그러나 사용자가 이런 목적으로 데이터의 변수를 줄이더라도 데이터가 원래 포함하고있던 정보가 줄어들면 곤란하다. 또한 데이터가 어떠한 모습을 가지고 있는지 쉽게 그래프로 그려보는 작업도 필요하다. 이와같이 데이터의 시각화를 편리하게 처리하기 위해서는 데이터의 차원, 즉 변수의 수가 줄어야만 한다. 너무 많은 데이터 변수들을 동시에 화면에 그리기는 어렵기 때문이다. 데이터 과학자에게는 위에 설명한 여러 가지 이유로 인하여 데이터의 [차원축소](https://en.wikipedia.org/wiki/Dimensionality_reduction)가 중요한 작업이 된다.

이번 예에서는 데이터 차원을 축소하는 방법의 하나로 선형차원 축소법을 소개하겠다. 이 알고리즘은 [Principal Component Analysis](https://en.wikipedia.org/wiki/Principal_component_analysis)로 알려져있는데 이를 사용하여, 보스턴 지역의 부동산 가격을 나타내는 데이터 중에 수치 데이터의 차원을 축소하는 예를 소개하겠다. 여기서는 [scikit-learn](http://scikit-learn.org/stable/index.html)라이브러리를 사용하겠다.

위키피디아에서 소개하는 [PCA를](https://en.wikipedia.org/wiki/Dimensionality_reduction#Principal_component_analysis) 아래 간단히 정리하였다.

---
차원을 축소하는 가장 널리 알려진 기본적인 방법은 PCA(principal component analysis)로서 주어진 데이터를 선형적으로 매핑하여 데이터의 차원을 줄이는 방법이며 차원이 축소된 데이터의 분산값이 최대가 되는 방향으로 축소한다. 데이터 간의 상관계수 또는 공분산(covariance) 매트릭스를 만들고 이 매트릭스의 고유벡터(eigenvector)들을 계산한다. 여기서 고유값(eigenvalues)가 큰 고유벡터들로 구성된, 차원이 축소된 벡터집합이 원래 벡터의 가장 큰 변화성향을 포함하고 있다고 가정하는 것이다. 이를 달리 표현하면 여러 변수중에서 변화 즉, 영향력이 큰 변수들의 순서대로 정리를 하고 영향력이 큰 변수들의 집합을 선택함으로써 차원을 줄이는 것이다. 차원이 축소된 데이터 세트는 원래 전체 데이터를 모두 사용하는 것에 비해서 정보를 일부 잃을 수도 있지만 데이터 분석 목적에는 크게 영향을 주지 않는 순서대로 변수를 줄임으로서 계산량을 줄이게 된다.

---

PCA를 이해하기 위해서 다음의 [시각화 자료](http://setosa.io/ev/principal-component-analysis/)와 [이문서](http://www.nature.com/nbt/journal/v26/n3/full/nbt0308-303.html)를 참고하기 바란다.

노트: PCA는 숫자로된 데이터 외에도 범주형 데이터에 대해서도 동작할 수 있으며 [이 예에서](http://stats.stackexchange.com/questions/5774/can-principal-component-analysis-be-applied-to-datasets-containing-a-mix-of-cont) [관련 기술](https://www.quora.com/Can-one-use-dimension-reduction-algorithms-like-PCA-for-categorical-variables) 특히 [PCA](https://www.ibm.com/support/knowledgecenter/SSLVMB_20.0.0/com.ibm.spss.statistics.help/idh_cpca.htm)에 관한 내용을 참고하기 바란다. 그러나 표준적인 PCA 알고리즘은 숫자 데이터에 대해서만 사용하는 것이 안전하다. 예에서 소개하는 보스턴 지역 부동산 데이터의 차원축소에서는 숫자로 된 데이터에 대해서만 처리를 하겠다 (즉, 범주형 데이터는 차원축소의 대상에서 제외한다).

In [37]:
from data.load_boston_housing import load_boston_housing
data = load_boston_housing()

# 숫자가 아닌 데이터 타입은 우선 제외한다. 원래 데이터를 백업했다가 필요한 부분을 복원하여 사용하겠다
# CMEDV는 수치데이터이지만 이는 차원축소에서 제외한다. 왜냐하면 이 데이터는 분석이 어떻게 동작하는지 파악하고자 
# 하는 대상 변수이기 때문이다
numerical_data = data.copy()
non_numerical_columns = ["OBS.", "TOWN", "TOWN#", "TRACT", "LON", "LAT", "MEDV", "CMEDV", "CHAS", "RAD"]
for column in non_numerical_columns:
    numerical_data = numerical_data.drop(column, 1)
    
print(numerical_data)

         CRIM    ZN  INDUS    NOX     RM    AGE     DIS  TAX  PTRATIO       B  \
0     0.00632  18.0   2.31  0.538  6.575   65.2  4.0900  296     15.3  396.90   
1     0.02731   0.0   7.07  0.469  6.421   78.9  4.9671  242     17.8  396.90   
2     0.02729   0.0   7.07  0.469  7.185   61.1  4.9671  242     17.8  392.83   
3     0.03237   0.0   2.18  0.458  6.998   45.8  6.0622  222     18.7  394.63   
4     0.06905   0.0   2.18  0.458  7.147   54.2  6.0622  222     18.7  396.90   
5     0.02985   0.0   2.18  0.458  6.430   58.7  6.0622  222     18.7  394.12   
6     0.08829  12.5   7.87  0.524  6.012   66.6  5.5605  311     15.2  395.60   
7     0.14455  12.5   7.87  0.524  6.172   96.1  5.9505  311     15.2  396.90   
8     0.21124  12.5   7.87  0.524  5.631  100.0  6.0821  311     15.2  386.63   
9     0.17004  12.5   7.87  0.524  6.004   85.9  6.5921  311     15.2  386.71   
10    0.22489  12.5   7.87  0.524  6.377   94.3  6.3467  311     15.2  392.52   
11    0.11747  12.5   7.87  

먼저 각 변수 (컬럼)의 값이 평균이 0 분산이 1이 되도록 스케일링 한다. 이렇게 하는 이유는 차원 축소를 하기 위해서 모든 변수의 크기를 동일한 조건으로 만들기 위해서이다. 이와 관련한 자세한 내용을 다음의 [문서](http://stats.stackexchange.com/a/105649)를 참조하기 바란다.

In [38]:
from sklearn import preprocessing
for column in numerical_data.columns:
    numerical_data[column] = preprocessing.scale(numerical_data[column])

print(numerical_data)

         CRIM        ZN     INDUS       NOX        RM       AGE       DIS  \
0   -0.419782  0.284830 -1.287909 -0.144217  0.413672 -0.120013  0.140214   
1   -0.417339 -0.487722 -0.593381 -0.740262  0.194274  0.367166  0.557160   
2   -0.417342 -0.487722 -0.593381 -0.740262  1.282714 -0.265812  0.557160   
3   -0.416750 -0.487722 -1.306878 -0.835284  1.016303 -0.809889  1.077737   
4   -0.412482 -0.487722 -1.306878 -0.835284  1.228577 -0.511180  1.077737   
5   -0.417044 -0.487722 -1.306878 -0.835284  0.207096 -0.351157  1.077737   
6   -0.410243  0.048772 -0.476654 -0.265154 -0.388411 -0.070229  0.839244   
7   -0.403696  0.048772 -0.476654 -0.265154 -0.160466  0.978808  1.024638   
8   -0.395935  0.048772 -0.476654 -0.265154 -0.931206  1.117494  1.087196   
9   -0.400729  0.048772 -0.476654 -0.265154 -0.399808  0.616090  1.329635   
10  -0.394346  0.048772 -0.476654 -0.265154  0.131589  0.914799  1.212979   
11  -0.406847  0.048772 -0.476654 -0.265154 -0.392685  0.509409  1.155935   

/usr/local/lib/python3.5/site-packages/sklearn/utils/validation.py:429: DataConversionWarning: Data with input dtype int64 was converted to float64 by the scale function.
  warnings.warn(msg, _DataConversionWarning)


In [39]:
from sklearn.decomposition import PCA

pca = PCA()
pca.fit(numerical_data)
print(pca.explained_variance_ratio_)

[ 0.50432371  0.11041929  0.10210929  0.07164118  0.0581276   0.04749511
  0.03581428  0.02005699  0.01836221  0.01684078  0.01480956]


위의 결과를 보면, 상대적인 분산 값을 비교하여 분산값이 큰 변수가 더 많은 정보를 포함하고 있다고 해석한다. 이제 얼마나 많은 변수를 축소하는 것이 타당한지를 결정해야 한다. 이를 위해 기준값(threshold)를 정해야 하는데 이 기준값 이상의 분산을 갖는 변수들만 선택한다. 이 기준값의 선택은 해당 데이터에 대한 이해와 주어진 문제에 따라 다르게 선택되어야 하며 어떤 절대적인 값이 있지는 않다. 상세한 분석을 하려면 가능한 많은 변수를 선택해야 하지만 불필요하게 시간이 오래걸리는 단점이 있다.

이 예에서는 임의로 기준값을 0.05로 선택하였다.

In [40]:
threshold = .05

num_above_threshold = len([value for value in pca.explained_variance_ratio_ if value >= threshold])
print("{} values are above the threshold {}".format(num_above_threshold, threshold))

5 values are above the threshold 0.05


위의 결과를 보면 기준값을 넘는 변수의 갯수가 5개인 것을 알 수 있다. 이제 이렇게 선택된 변수들과, 초기에 차원축소에서 제외했던 변수들을 다시 합하여 데이터 분석에서 사용할 데이터 셋을 구성하면 된다.

In [41]:
pca.n_components = num_above_threshold
numerical_data_reduced = pca.fit_transform(numerical_data)
print(numerical_data_reduced)

[[-1.86750035  1.11108108  0.28008935  0.56875387  0.07391108]
 [-1.21419895  0.50592299 -0.70293321 -0.10301129 -0.32320405]
 [-1.88588902  0.95929021 -0.04363222 -0.93524529 -0.33569258]
 ..., 
 [ 0.02402412  1.23443503 -0.60519005 -1.62266184 -0.14603066]
 [ 0.0698065   1.0195971  -0.71288554 -1.44770355 -0.19010325]
 [ 0.23171772  0.3718647  -1.2479854  -0.86550239 -0.22855798]]


지금까지 11개의 변수를 5개로 줄이는 과정을 설명했다. 처음에 차원축소에서 제외했던 변수들을 포함하여 새로운 데이터 세트를 구성하고 이를 사용하여 데이터를 분석하면 된다.